# Database tool to store portfolio data for DCA strategy

Install and create the SQLite DB

In [1]:
!pip install ipython-sql

In [2]:
%load_ext sql

In [3]:
%sql sqlite://

In [4]:
import pandas as pd
import sqlite3

Create a table and DB (if not created yet)

In [5]:
def create_table():
    conn = sqlite3.connect('portfolio.db')
    c = conn.cursor()
    try:
        # actual value (from last input POV)
        c.execute("""
            CREATE TABLE DCA_portfolio (
            company_name text,
            ticker text,
            shares integer,
            last_buy_price real,
            avg_price real,
            invested_value real,
            actual_value real,
            profit real,
            share_in_portfolio real
            )""")
        conn.commit()
        conn.close()
    except Exception as e:
        conn.commit()
        conn.close()
        print(e)

In [6]:
create_table()

Insert some test data statically

In [7]:
def insert_test_data():
    ''' Used just for initial testing '''
    conn = sqlite3.connect('portfolio.db')
    c = conn.cursor()
    c.execute("INSERT INTO DCA_portfolio VALUES (?,?,?,?,?,?,?,?,?)", ('company_name', 'xxx', 100, 50.2, 48.25, 4825, 5020, 195, 1))
    c.execute("INSERT INTO DCA_portfolio VALUES (?,?,?,?,?,?,?,?,?)", ('company_name2', 'yyy', 75, 102.5, 102.5, 7687.5, 7687.5, 0, 1))
    conn.commit()
    conn.close()

In [8]:
# insert_test_data()

Show the portfolio with test data

In [10]:
def show_portfolio():
    conn = sqlite3.connect('portfolio.db')
    c = conn.cursor()
    c.execute("SELECT * FROM DCA_portfolio")
    table = c.fetchall()
    conn.commit()
    conn.close()
    return table

In [11]:
show_portfolio()

[('company_name', 'xxx', 100, 50.2, 48.25, 4825.0, 5020.0, 195.0, 1.0),
 ('company_name2', 'yyy', 75, 102.5, 102.5, 7687.5, 7687.5, 0.0, 1.0)]

Create the functions needed to calculate values for new portfolio position

In [12]:
def get_last_buy_price(c, ticker, price):
    c.execute("SELECT last_buy_price FROM DCA_portfolio WHERE ticker = ?", (ticker,))
    lbp = c.fetchall()
    return lbp[0][0] if lbp else price

In [13]:
def get_avg_price(c, ticker, price, shares):
    c.execute("SELECT avg_price FROM DCA_portfolio WHERE ticker = ?", (ticker,))
    ap = c.fetchall()
    if not ap:
        return price
    else:
        c.execute("SELECT shares FROM DCA_portfolio WHERE ticker = ?", (ticker,))
        ash = c.fetchall()[0][0]
        ap = ap[0][0]
        ap = (ap * ash + shares * price)/(ash+shares)
        return ap

In [14]:
# get invested value
def get_invested_value(c, ticker, price, shares):
    c.execute("SELECT invested_value FROM DCA_portfolio WHERE ticker = ?", (ticker,))
    iv = c.fetchall()
    if not iv:
        return price*shares
    else:
        return iv[0][0]+(price*shares)

In [15]:
# get actual value (from last input POV)
def get_actual_value(c, ticker, price, shares):
    c.execute("SELECT actual_value FROM DCA_portfolio WHERE ticker = ?", (ticker,))
    av = c.fetchall()
    if not av:
        return price*shares
    else:
        c.execute("SELECT shares FROM DCA_portfolio WHERE ticker = ?", (ticker,))
        ash = c.fetchall()[0][0]
        return price*(shares+ash)

In [16]:
def get_share_in_portfolio(c, actual_value):
    ''' get the current size of the position in the portfolio [%] '''
    c.execute("SELECT actual_value FROM DCA_portfolio")
    psvl = c.fetchall()
    if not psvl:
        return 100
    else:
        avl = [x[0] for x in psvl]
        return 100*actual_value/(sum(avl)+actual_value) # sum of existing + the new position

In [17]:
def get_shares(c, ticker, shares):
    c.execute("SELECT shares FROM DCA_portfolio WHERE ticker=?", (ticker,))
    return c.fetchall()[0][0]+shares

In [18]:
def asset_exist(company_name, ticker):
    conn = sqlite3.connect('portfolio.db')
    c = conn.cursor()
    c.execute("SELECT company_name, ticker FROM DCA_portfolio WHERE company_name=? AND ticker=?", (company_name, ticker))
    exist = c.fetchone()
    conn.commit()
    conn.close()
    return bool(exist)

In [19]:
def round_2(n):
    return round(n, 2)

Function for adding new position into portfolio

In [20]:
# would be better to perform it as a class (along with adding functions)
def add_shares(company_name, ticker, shares, price):
    conn = sqlite3.connect('portfolio.db')
    c = conn.cursor()
    
    last_buy_price = price #get_last_buy_price(c, ticker, price)
    avg_price = round_2(get_avg_price(c, ticker, price, shares))
    invested_value = round_2(get_invested_value(c, ticker, price, shares))
    actual_value = round_2(get_actual_value(c, ticker, price, shares))
    profit = round_2(100*(actual_value-invested_value)/invested_value)
    share_in_portfolio = round_2(get_share_in_portfolio(c, actual_value)) # that will need to be updated (the values to be wrong after adding new asset)
    
    if asset_exist(company_name, ticker):
        shares = get_shares(c, ticker, shares)
        c.execute("""UPDATE DCA_portfolio 
                  SET shares=?,
                      last_buy_price=?,
                      avg_price=?,
                      invested_value=?,
                      actual_value=?,
                      profit=?,
                      share_in_portfolio=?
                  WHERE company_name=? AND ticker=?""", (shares, last_buy_price, avg_price, invested_value, actual_value, profit, share_in_portfolio, company_name, ticker))
    else:
        c.execute("INSERT INTO DCA_portfolio VALUES (?,?,?,?,?,?,?,?,?)", (company_name, ticker, shares, last_buy_price, avg_price, invested_value, actual_value, profit, share_in_portfolio))

    conn.commit()
    conn.close()

Let's add a test asset

In [21]:
add_shares('test', 't2', 200, 37.5)

In [22]:
show_portfolio()

[('company_name', 'xxx', 100, 50.2, 48.25, 4825.0, 5020.0, 195.0, 1.0),
 ('company_name2', 'yyy', 75, 102.5, 102.5, 7687.5, 7687.5, 0.0, 1.0),
 ('test', 't2', 200, 37.5, 37.5, 7500.0, 7500.0, 0.0, 37.11)]

Let's update asset

In [23]:
add_shares('company_name', 'xxx', 50, 51.2)
add_shares('company_name2', 'yyy', 100, 97.5)

In [24]:
show_portfolio()

[('company_name', 'xxx', 150, 51.2, 49.23, 7385.0, 7680.0, 3.99, 27.54),
 ('company_name2', 'yyy', 175, 97.5, 99.64, 17437.5, 17062.5, -2.15, 42.73),
 ('test', 't2', 200, 37.5, 37.5, 7500.0, 7500.0, 0.0, 37.11)]

After adding at least one asset the share_in_portfolio column needs to be updated

In [28]:
def update_the_share_in_portfolio():
    conn = sqlite3.connect('portfolio.db')
    c = conn.cursor()
    c.execute("SELECT company_name, actual_value FROM DCA_portfolio") # instead of the company name the ticker can be used as well
    psvl = c.fetchall()
    avs = sum([x[1] for x in psvl])
    for comp, value in psvl:
        c.execute("UPDATE DCA_portfolio SET share_in_portfolio = ? WHERE company_name = ?", (round_2(100*value/(avs)), comp))
    conn.commit()
    conn.close()

In [29]:
update_the_share_in_portfolio()

Let's verify the updates

In [30]:
show_portfolio()

[('company_name', 'xxx', 150, 51.2, 49.23, 7385.0, 7680.0, 3.99, 23.82),
 ('company_name2', 'yyy', 175, 97.5, 99.64, 17437.5, 17062.5, -2.15, 52.92),
 ('test', 't2', 200, 37.5, 37.5, 7500.0, 7500.0, 0.0, 23.26)]